LOAD STUPID GT AND BBOX PREDICTIONS

In [1]:
import cPickle
import numpy as np
import cv2

In [2]:
gt_pkl = '/home/lioruzan/car_detection_proj/py-faster-rcnn/data/OIRDS_v1_0/OIRDS_test.pkl'
det_pkl = '/home/lioruzan/car_detection_proj/py-faster-rcnn/output/faster_rcnn_alt_opt/cars_test/cars_VGG16_faster_rcnn_final/nms_detections.pkl'

with open(gt_pkl, 'rb') as f:
    gt_test = cPickle.load(f)
    for k,v in gt_test.iteritems():
        gt_test[k] = np.array(v)
with open(det_pkl, 'rb') as f:
    det_test = cPickle.load(f)
    det_test = det_test[1]

In [3]:
img_list = sorted(gt_test.keys())

SMOOSH THEM TOGETHER PER IMAGE

In [18]:
import string
import random

def vis_detections(im, tp, fp, fn):
    """Visual debugging of detections."""
    import matplotlib.pyplot as plt
    im = im[:, :, (2, 1, 0)]
    plt.cla()
    plt.imshow(im)
    _draw_detections(im, tp, fp, fn, plt)
    plt.show()

def save_vis_detections(idx, out_dir, im, tp, fp, fn):
    import matplotlib.pyplot as plt
    im = im[:, :, (2, 1, 0)]
    plt.cla()
    plt.imshow(im)
    _draw_detections(im, tp, fp, fn, plt)
    plt.savefig(osp.join(out_dir,'{:04}.png'.format(idx)))

def _draw_detections(im, tp, fp, fn, plt):
    for i in xrange(tp.shape[0]):
        bbox = tp[i, :4]
        plt.gca().add_patch(
            plt.Rectangle((bbox[0], bbox[1]),
                          bbox[2] - bbox[0],
                          bbox[3] - bbox[1], fill=False,
                          edgecolor='green', linewidth=2)
            )
    for i in xrange(fp.shape[0]):
        bbox = fp[i, :4]
        plt.gca().add_patch(
            plt.Rectangle((bbox[0], bbox[1]),
                          bbox[2] - bbox[0],
                          bbox[3] - bbox[1], fill=False,
                          edgecolor='red', linewidth=2)
            )
    for i in xrange(fn.shape[0]):
        bbox = fn[i, :4]
        plt.gca().add_patch(
            plt.Rectangle((bbox[0], bbox[1]),
                          bbox[2] - bbox[0],
                          bbox[3] - bbox[1], fill=False,
                          edgecolor='yellow', linewidth=2)
            )
    return
    
# gets IoUs for a single pic
def get_ious(dt, gt):
    """
    output 
        -a DxG grid of IoUs (D = num of detections, G = num of ground truths)
        -score vector for said detections
    """ 
    ious = np.zeros((len(dt),len(gt)), dtype=np.float32)
    x1_dt, x1_gt = dt[:, 0], gt[:, 0]
    y1_dt, y1_gt = dt[:, 1], gt[:, 1]
    x2_dt, x2_gt = dt[:, 2], gt[:, 2]
    y2_dt, y2_gt = dt[:, 3], gt[:, 3]
    
    areas_dt = (x2_dt - x1_dt + 1) * (y2_dt - y1_dt + 1)
    areas_gt = (x2_gt - x1_gt + 1) * (y2_gt - y1_gt + 1)
    
    for i in xrange(len(dt)):
        xx1 = np.maximum(x1_dt[i], x1_gt)
        yy1 = np.maximum(y1_dt[i], y1_gt)
        xx2 = np.minimum(x2_dt[i], x2_gt)
        yy2 = np.minimum(y2_dt[i], y2_gt)
        w = np.maximum(0.0, xx2 - xx1 + 1)
        h = np.maximum(0.0, yy2 - yy1 + 1)
        
        inter = w * h
        ovr = inter / (areas_dt[i] + areas_gt - inter)
        ious[i] = ovr
    return ious

def sort_dets(dets, gt, iou_thresh=0.5):
    tp = []
    fp = []
    fn = []
    ious = get_ious(dets, gt)
    gt_idx = range(gt.shape[0])
    
    for j in xrange(dets.shape[0]):
        current_ious = ious[j, gt_idx]
        
        # no more objects -> count as fp
        if current_ious.shape[0] == 0:
            fp.append(dets[j])
            continue
            
        best = np.argmax(current_ious)
        if current_ious[best] >= iou_thresh:
            tp.append(dets[j])
            gt_idx.pop(best)
        else:
            fp.append(dets[j])
    # remaining unclaimed objects are false negatives    
    remaining_gt = ious[:, gt_idx]
    if remaining_gt.shape[1] > 0:
        fn = gt[gt_idx, :]
        
    return np.array(tp), np.array(fp), np.array(fn)

FIX BUG AND INTRODUCE SCORE THRESHOLD

In [22]:
import os.path as osp
root = '/home/lioruzan/car_detection_proj/py-faster-rcnn/data/OIRDS_v1_0'
out_dir = '/home/lioruzan/Desktop/idf_slides/det_examples'
# limit to N top dets
N = 5
for j in xrange(len(img_list)):
    im = cv2.imread(osp.join(root, img_list[j]))
    dt = det_test[j][:N]
    gt = gt_test[img_list[j]]
    tp, fp, fn = sort_dets(dt, gt, iou_thresh=0.5)
    save_vis_detections(j, out_dir, im, tp, fp, fn)
#     vis_detections(im, tp, fp, fn)
    

In [51]:
fn

array([[168,   1, 187,  13]])